In [37]:
from biodivine_aeon import *
from pathlib import Path

In [38]:
model = BooleanNetwork.from_file("model_map1.aeon")
print(model)

BooleanNetwork(variables=91, regulations=124, explicit_parameters=37, implicit_parameters=22)


In [39]:
stg = AsynchronousGraph(model)

First, we identify the unknown functions that can be safely eliminated because they have only one interpretations that satisfies all the requirements regarding their input monotonicity and essentiality.

In [40]:
to_eliminate = []
for param in model.explicit_parameters():
    print(model.get_explicit_parameter_name(param))
    count = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    if count == 1:
        to_eliminate.append(param)

cat_Fc_gamma_RIIB_phosphorylated_66
 >  ColorModel({'cat_Fc_gamma_RIIB_phosphorylated_66': 'x_0'})
cat_MEK1_MAP2K1__phosphorylated_50
 >  ColorModel({'cat_MEK1_MAP2K1__phosphorylated_50': 'x_0'})
cat_IKK_Beta_phosphorylated_39
 >  ColorModel({'cat_IKK_Beta_phosphorylated_39': 'x_0'})
cat_DAG_simple_molecule_65
 >  ColorModel({'cat_DAG_simple_molecule_65': '(x_0 & x_1)'})
 >  ColorModel({'cat_DAG_simple_molecule_65': '((!x_0 & x_1) | x_0)'})
cat_Fos_phosphorylated_68
 >  ColorModel({'cat_Fos_phosphorylated_68': 'x_0'})
cat_JNK_phosphorylated_70
 >  ColorModel({'cat_JNK_phosphorylated_70': '(x_0 & x_1)'})
 >  ColorModel({'cat_JNK_phosphorylated_70': '((!x_0 & x_1) | x_0)'})
cat_MEK2_MAP2K2__phosphorylated_41
 >  ColorModel({'cat_MEK2_MAP2K2__phosphorylated_41': 'x_0'})
cat_PLCG1_phosphorylated_64
 >  ColorModel({'cat_PLCG1_phosphorylated_64': 'x_0'})
cat_FceRI_Syk_complex_Cytosol_1_25
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_25': 'x_0'})
cat_Elk1_phosphorylated_53
 >  ColorModel(

In [41]:
print(to_eliminate)

[ParameterId(10), ParameterId(23), ParameterId(17), ParameterId(14), ParameterId(24), ParameterId(31), ParameterId(12), ParameterId(19), ParameterId(26), ParameterId(28), ParameterId(3), ParameterId(36), ParameterId(15), ParameterId(1), ParameterId(32), ParameterId(2), ParameterId(0), ParameterId(4), ParameterId(13), ParameterId(33), ParameterId(30), ParameterId(6), ParameterId(21), ParameterId(25), ParameterId(16), ParameterId(27)]


In [42]:
m = next(stg.mk_unit_colors().items(retained = to_eliminate))

In [43]:
model_simplified = m.instantiate(model)

PanicException: assertion failed: translation_map.contains_key(&var) || param_map.contains_key(&var)

In [ ]:
Path("model_map1.simplified.aeon").write_text(model_simplified.to_aeon())

In [ ]:
stg = AsynchronousGraph(model_simplified)

In [ ]:
# This identifies model outputs. In the next step, we restrict them manually to
# a subset of outputs that are biologically interpretable.
outputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.successors(var)) == 0:
        outputs.append(var)
outputs

In [ ]:
# Actually relevant outputs:
outputs = ['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Jun_Fos_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex']
outputs

In [ ]:
inputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.predecessors(var)) == 0:
        inputs.append(var)
inputs

In [ ]:
params = []
for p in model_simplified.explicit_parameter_names():
    params.append((p, model_simplified.get_explicit_parameter_arity(p)))
params

In [ ]:
ctx = SymbolicSpaceContext(model_simplified)
stg = AsynchronousGraph(model_simplified, ctx)

Compute minimal trap spaces across all interpretations. These should almost exactly correspond to model attractors (this can take a few minutes).

In [ ]:
traps = TrapSpaces.essential_symbolic(ctx, stg, ctx.mk_unit_colored_spaces(stg))

In [ ]:
min_traps = TrapSpaces.minimize(ctx, traps)

In [ ]:
min_traps

Compute the number of output value combinations that can actually appear in the model, and the distribution of output values within these combinations.

In [ ]:
phenotype_iterator = min_traps.spaces().items(retained = outputs)
phenotype_count = sum(1 for _ in phenotype_iterator)
print(f"Phenotypes: {phenotype_count}")

In [ ]:
is_zero = { var: 0 for var in outputs }
is_one = { var: 0 for var in outputs }
is_any = { var: 0 for var in outputs }
for phenotype in min_traps.spaces().items(retained = outputs):
    phenotype_traps = min_traps.intersect_spaces(phenotype.to_symbolic())
    for (var, value) in phenotype.items():        
        var = ctx.get_network_variable_name(var)
        if value == True:
            is_one[var] += 1
        elif value == False:
            is_zero[var] += 1
        else:
            is_any[var] += 1

print("Zero:", is_zero)
print("One:", is_one)
print("Any:", is_any)

In [ ]:
def cardinality_without_inputs(ctx, inputs, set):
    # Project the BDD to remove inputs.
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    all_vars = ctx.bdd_variable_set().variable_count()
    function_vars = sum(len(x) for x in ctx.explicit_functions_bdd_variables().values())

    general_cardinality = set_bdd.cardinality()
    return int(general_cardinality / (2 ** (all_vars - function_vars)))

def prune_inputs(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    return ColorSet(ctx, set_bdd)

def prune_inputs_forall(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_for_all(ignore_vars)

    return ColorSet(ctx, set_bdd)

Build a table showing the multiplicity of various attractor combinations for each output variable:

In [ ]:
print(f"0\t1\t*\t0-1\t1-*\t0-*\t0-1-*\tname")

for var in outputs:
    p_var = ctx.get_positive_space_variable(var)
    n_var = ctx.get_negative_space_variable(var)
    var_is_one = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: False }))
    var_is_zero = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: False, n_var: True }))
    var_is_any = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: True }))

    one_colors = min_traps.intersect_spaces(var_is_one).colors()
    zero_colors = min_traps.intersect_spaces(var_is_zero).colors()
    any_colors = min_traps.intersect_spaces(var_is_any).colors()

    # one_colors = prune_inputs(ctx, inputs, one_colors)
    # zero_colors = prune_inputs(ctx, inputs, zero_colors)
    # any_colors = prune_inputs(ctx, inputs, any_colors)

    one = one_colors.minus(zero_colors).minus(any_colors)
    zero = zero_colors.minus(one_colors).minus(any_colors)
    any = any_colors.minus(zero_colors).minus(one_colors)
    zero_one = one_colors.intersect(zero_colors).minus(any_colors)
    one_any = one_colors.intersect(any_colors).minus(zero_colors)
    zero_any = zero_colors.intersect(any_colors).minus(one_colors)
    zero_one_any = zero_colors.intersect(one_colors).intersect(any_colors)
    
    # c_one = cardinality_without_inputs(ctx, inputs, one)
    # c_zero = cardinality_without_inputs(ctx, inputs, zero)
    # c_any = cardinality_without_inputs(ctx, inputs, any)
    # c_zero_one = cardinality_without_inputs(ctx, inputs, zero_one)
    # c_one_any = cardinality_without_inputs(ctx, inputs, one_any)
    # c_zero_any = cardinality_without_inputs(ctx, inputs, zero_any)
    # c_zero_one_any = cardinality_without_inputs(ctx, inputs, zero_one_any)

    c_one = one.cardinality()
    c_zero = zero.cardinality()
    c_any = any.cardinality()
    c_zero_one = zero_one.cardinality()
    c_one_any = one_any.cardinality()
    c_zero_any = zero_any.cardinality()
    c_zero_one_any = zero_one_any.cardinality()

    print(f"{c_zero}\t{c_one}\t{c_any}\t{c_zero_one}\t{c_one_any}\t{c_zero_any}\t{c_zero_one_any}\t{var}")    

Finally, let us compare the partial network to the original CaSQ network:

In [ ]:
model_known = BooleanNetwork.from_file("model_map1.full.aeon")
print(model_known)

In [ ]:
ctx_known = SymbolicSpaceContext(model_known)
stg_known = AsynchronousGraph(model_known, ctx_known)

In [ ]:
traps_known = TrapSpaces.essential_symbolic(ctx_known, stg_known, ctx_known.mk_unit_colored_spaces(stg_known))

In [ ]:
min_traps_known = TrapSpaces.minimize(ctx_known, traps_known)

In [ ]:
min_traps_transferred = ColoredSpaceSet(ctx, ctx.transfer_from(min_traps_known.to_bdd(), ctx_known))

In [ ]:
equivalent = prune_inputs_forall(ctx, inputs, min_traps.intersect(min_traps_transferred).colors())
print("All models:", cardinality_without_inputs(ctx, inputs, stg.mk_unit_colors()))
print("Equivalent models:", cardinality_without_inputs(ctx, inputs, equivalent))

Let's examine which funtions are fixed/free in the 128 equivalent models:

In [ ]:
for param in model_simplified.explicit_parameters():
    print(model_simplified.get_explicit_parameter_name(param))
    count = 0
    for m in equivalent.items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    total = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        total += 1
    print(f"{count}/{total}")